In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

def which_taxo(file):

    if (file.find("a.") != -1):
        taxo = pd.read_csv("test/taxoS_test.csv")
        nom = "CRC Full S"
    elif (file.find("b.") != -1):
        taxo = pd.read_csv("test/taxoS1_test.csv")
        nom = "CRC Full S1"
    elif (file.find("k.") != -1):
        taxo = pd.read_csv("test/taxoS_test.csv")
        nom = "CRC Full + Regió S"
    elif (file.find("l.") != -1):
        taxo = pd.read_csv("test/taxoS1_test.csv")
        nom = "CRC Full + Regió S1"
    elif (file.find("m.") != -1):
        taxo = pd.read_csv("test/taxoS_test.csv")
        nom = "CRC Full + Seq_tool S"
    elif (file.find("n.") != -1):
        taxo = pd.read_csv("test/taxoS1_test.csv")
        nom = "CRC Full + Seq_tool S1"
    elif (file.find("o.") != -1):
        taxo = pd.read_csv("test/taxoS_test.csv")
        nom = "CRC Full + Regió + Seq_tool S"
    elif (file.find("p.") != -1):
        taxo = pd.read_csv("test/taxoS1_test.csv")
        nom = "CRC Full + Regió + Seq_tool S1"
    return taxo, nom

def which_meta_extra(file):
    if (file.find("m.") != -1) or (file.find("n.") != -1):
        meta_extra = ["seq_tool"]
    
    elif (file.find("o.") != -1) or (file.find("p.") != -1):
        meta_extra = ["region","seq_tool"]
        
    elif (file.find("k.") != -1) or (file.find("l.") != -1):
        meta_extra = ["region"]
        
    else:
        meta_extra = []
    return meta_extra

def filt_y_rows(taxo,meta):
    condition = []
    for i in range(len(taxo)):
        condition.append(pd.DataFrame(meta["condition"][meta["sample"] == 
                                                        taxo["sample"][i]]).iloc[0,0])
    return condition

def add_metas(taxo,meta,meta_extra=[]):
    for i in meta_extra:
        meta_extra_i = []
        for j in range(len(taxo)):
            meta_extra_i.append(pd.DataFrame(meta[i][meta["sample"] ==
                                          taxo["sample"][j]]).iloc[0,0])
        taxo[i] = meta_extra_i
    return taxo

def drop_nas(taxo,meta):
    taxo["condition"] = meta
    taxo = taxo.dropna()
    meta = taxo["condition"]
    taxo = taxo.iloc[:,1:-1]
    return taxo, meta

def get_roc_curves(y_test, y_prob, pos_label):
    fper1, tper1, tresholds1 = roc_curve(y_test, y_prob, pos_label=pos_label)
    fig,base = plt.subplots()
    base.set_title("Corba ROC" + " " + nomi)
    base.set_xlabel("Falsos positius")
    base.set_ylabel("Sensibilitat")
    base.plot(fper1, tper1)
    base.plot([0,1],[0,1],'-')
    fig.savefig("metrics/" + file + '_roc_curve.png')
    plt.close()

def get_test_metrics(model, x_test, y_test, labels):
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:,1]
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels = labels).ravel()
    acc = accuracy_score(y_test, y_pred)
    sens = tp / (tp+fn)
    espe = tn / (tn+fp)
    AUC = roc_auc_score(y_test, y_prob)
    get_roc_curves(y_test, (1 - y_prob), labels[1])
    return acc, sens, espe, AUC



In [2]:
DirExists = os.path.exists("metrics")
if not DirExists:
    os.makedirs("metrics")
nom = []
nom_model = []
N = []
acc = []
sens = []
espe = []
AUC = []
for file in os.listdir("models/mipredictor/"):
    print(file)
    x_test, nomi = which_taxo(file)
    y = pd.read_csv("metadades_full.csv")
    meta_extra = which_meta_extra(file)
    y_test = filt_y_rows(x_test, y)
    x_test = add_metas(x_test, y, meta_extra)
    x_test, y_test = drop_nas(x_test, y_test)
    model = load("models/mipredictor/" + file)
    nom_model.append(str(model))
    N.append(len(x_test))
    nom.append(nomi)
    acci, sensi, espei, AUCi =  get_test_metrics(model, x_test, y_test, ["Control","CRC"])
    acc.append(acci)
    sens.append(sensi)
    espe.append(espei)
    AUC.append(AUCi)

results = pd.DataFrame()
results["Nom"] = nom
results["Model"] = nom_model
results["n (test)"] = N
results["Exactitud"] = acc
results["Sensibilitat"] = sens
results["Especificitat"] = espe
results["AUC"] = AUC

results.to_csv("metrics/metriques_mipredictor.csv", index=False)

print("metrics done")

a.joblib


C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


b.joblib


C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\46375547A\

k.joblib


...

C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


o.joblib


C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


p.joblib
metrics done


C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\46375547A\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\46375547A\

In [3]:
results

,Nom,Model,n (test),Exactitud,Sensibilitat,Especificitat,AUC
0,CRC Full S,DecisionTreeClassifier(),45,0.622222,0.521739,0.727273,0.612648
1,CRC Full S1,RandomForestClassifier(n_estimators=10),45,0.600000,0.565217,0.636364,0.584980
2,CRC Full + Regió S,DecisionTreeClassifier(),45,0.622222,0.521739,0.727273,0.612648
3,CRC Full + Regió S1,RandomForestClassifier(n_estimators=10),45,0.555556,0.347826,0.772727,0.620553
4,CRC Full + Seq_tool S,DecisionTreeClassifier(),33,0.515152,0.473684,0.571429,0.565789
5,CRC Full + Seq_tool S1,DecisionTreeClassifier(),33,0.515152,0.473684,0.571429,0.535714
6,CRC Full + Regió + Seq_tool S,DecisionTreeClassifier(),33,0.575758,0.578947,0.571429,0.543233
7,CRC Full + Regió + Seq_tool S1,RandomForestClassifier(n_estimators=10),33,0.393939,0.473684,0.285714,0.469925
